# Gemini models in  SAP AI Core

Using direct API calls with Vertex AI format.

## 1. Setup

In [19]:
import os
import json
import requests


## 2. Set Credentials

In [33]:
# Path to your AI Core service key JSON file
service_key_path = "../aicore-service-key.json"

# Load service key
with open(service_key_path, 'r') as f:
    service_key = json.load(f)

# Set environment variables
os.environ['AICORE_AUTH_URL'] = service_key['url']
os.environ['AICORE_CLIENT_ID'] = service_key['clientid']
os.environ['AICORE_CLIENT_SECRET'] = service_key['clientsecret']
os.environ['AICORE_RESOURCE_GROUP'] = 'default'

# Parse serviceurls - it might be a string or already a dict
serviceurls = service_key['serviceurls']
if isinstance(serviceurls, str):
    serviceurls = json.loads(serviceurls)
os.environ['AICORE_BASE_URL'] = serviceurls['AI_API_URL']
base_url = os.environ['AICORE_BASE_URL']

print("✓ AI Core credentials configured")


✓ AI Core credentials configured


## 3. Get Auth Token

In [27]:
from gen_ai_hub.proxy.core.proxy_clients import get_proxy_client

# Get auth headers
proxy = get_proxy_client('gen-ai-hub')
headers = proxy.get_request_header()

print('✓ Auth token obtained')


✓ Auth token obtained


## 3b. Check for Image Generation Models


In [29]:
# List all deployments and check for image generation models
deployments = proxy.get_deployments()

print('All Available Deployments:')
print('='*80)

image_models = []

for d in deployments:
    model_name = d.model_name.lower()
    if "gemini" in model_name:
        print(f"\nModel: {d.model_name}")
        print(f"  Deployment ID: {d.deployment_id}")
        
        # Check if it's an image generation model
        if any(x in model_name for x in ['dall-e', 'dalle', 'stable-diffusion', 'sd-', 'imagen']):
            image_models.append(d)
            print("  ✓ IMAGE GENERATION MODEL")
        else:
            
            print("  📝 Text/Analysis model")

    print('\n' + '='*80)
    print(f'\nSummary: {len(image_models)} image generation model(s) found')

    if image_models:
        print('\n✓ Image generation models:')
        for m in image_models:
            print(f'  - {m.model_name} (ID: {m.deployment_id})')
    else:
        print('\n⚠️  No image generation models deployed in your SAP AI Core')


All Available Deployments:


Summary: 0 image generation model(s) found

⚠️  No image generation models deployed in your SAP AI Core

Model: gemini-2.5-pro
  Deployment ID: d6ae523ed14c6cc3
  📝 Text/Analysis model


Summary: 0 image generation model(s) found

⚠️  No image generation models deployed in your SAP AI Core

Model: gemini-2.0-flash
  Deployment ID: d83ad32cbda0399d
  📝 Text/Analysis model


Summary: 0 image generation model(s) found

⚠️  No image generation models deployed in your SAP AI Core


Summary: 0 image generation model(s) found

⚠️  No image generation models deployed in your SAP AI Core


Summary: 0 image generation model(s) found

⚠️  No image generation models deployed in your SAP AI Core


Summary: 0 image generation model(s) found

⚠️  No image generation models deployed in your SAP AI Core


Summary: 0 image generation model(s) found

⚠️  No image generation models deployed in your SAP AI Core


Summary: 0 image generation model(s) found

⚠️  No image generati

## 4. Call Gemini 2.5 Pro (Vertex AI Format)

In [34]:
# Gemini 2.5 Pro deployment
DEPLOYMENT_ID = 'd6ae523ed14c6cc3'
url = f'{base_url}/v2/inference/deployments/{DEPLOYMENT_ID}/models/gemini-2.5-pro:generateContent'

# Vertex AI format payload
payload = {
    'contents': [
        {
            'role': 'user',
            'parts': [{'text': 'can you generate the cat image'}]
        }
    ],
    'generationConfig': {
        'temperature': 0.7,
        'maxOutputTokens': 10000
    }
}

response = requests.post(url, headers=headers, json=payload)

if response.status_code == 200:
    result = response.json()
    print('✓ Success!')
    print(result)
    print(f"\nResponse:\n{result['candidates'][0]['content']['parts'][0]['text']}")
else:
    print(f'Error {response.status_code}: {response.text}')


✓ Success!
{'candidates': [{'avgLogprobs': -0.6888435347908004, 'content': {'parts': [{'text': 'Of course!\n\nAs a text-based AI, I can\'t generate a visual image file directly. However, I can do the next best thing! I can either:\n\n1.  **Describe a cat image in detail** for you to use with an AI image generator (like Midjourney, DALL-E, or Stable Diffusion).\n2.  **Create a cat for you using text characters** (ASCII art).\n\nWhich would you prefer? Here are both options for you!\n\n---\n\n### 1. Detailed Descriptions for an AI Image Generator\n\nCopy and paste one of these prompts into an AI image tool to get a fantastic picture.\n\n**Prompt for a Photorealistic Cat:**\n> A highly detailed, photorealistic close-up of a fluffy Siberian cat sleeping in a patch of sunlight on a wooden floor. The cat\'s long, silver-tabby fur is soft and distinct. Dust motes dance in the warm, golden sunbeam. The focus is sharp on the cat\'s face, showing its peaceful expression and twitching whiskers. T

## 4b. Test Image Generation with Gemini 2.0 Flash


In [35]:
# Test if Gemini 2.0 Flash can generate images
DEPLOYMENT_ID_2_0 = 'd83ad32cbda0399d'
url = f'{base_url}/v2/inference/deployments/{DEPLOYMENT_ID_2_0}/models/gemini-2.0-flash:generateContent'

# Try to ask it to generate an image
payload = {
    'contents': [
        {
            'role': 'user',
            'parts': [{'text': 'can you generate a cat image'}]
        }
    ],
    'generationConfig': {
        'temperature': 0.7,
        'maxOutputTokens': 1000
    }
}

response = requests.post(url, headers=headers, json=payload)

if response.status_code == 200:
    result = response.json()
    print('✓ Success!')
    print(f"\nResponse:\n{result['candidates'][0]['content']['parts'][0]['text']}")
else:
    print(f'Error {response.status_code}: {response.text}')
    
print('\n' + '='*80)
print('Note: Gemini models (both 2.5 Pro and 2.0 Flash) can ANALYZE images')
print('but cannot GENERATE images. They are text and image understanding models.')
print('For image generation, you would need models like DALL-E or Stable Diffusion.')


✓ Success!

Response:
I am unable to generate images directly. I am a text-based AI.

However, I can give you some ideas of what to search for online, or suggest prompts for an image generation tool if you have access to one. For example:

**Ideas for Image Searches:**

*   "Cute kitten"
*   "Sleeping cat"
*   "Funny cat meme"
*   "Cat portrait"
*   "Cartoon cat"

**Prompts for Image Generation Tools:**

*   "A fluffy ginger cat sitting in a sunbeam"
*   "A sleek black cat with green eyes staring intensely"
*   "A whimsical illustration of a cat wearing a tiny hat"
*   "A photorealistic image of a tabby cat curled up on a couch"
*   "A pixel art cat sprite"

I hope this helps you find the purr-fect cat image! 🐱


Note: Gemini models (both 2.5 Pro and 2.0 Flash) can ANALYZE images
but cannot GENERATE images. They are text and image understanding models.
For image generation, you would need models like DALL-E or Stable Diffusion.
